In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import numpy as np 
import pandas as pd 
import os
from tqdm import tqdm
import csv
import cv2
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator


train_dataset = pd.read_csv("../input/com3025-2022-image-classification-challenge/train.csv")

# resize image to feed into model
def read_and_resize_image(image_path):
    im = cv2.imread(image_path) 
    im_resized = cv2.resize(im, (224, 224), interpolation=cv2.INTER_LINEAR)
    im_converted = cv2.cvtColor(im_resized, cv2.COLOR_BGR2RGB)
    return im_converted
import matplotlib.pyplot as plt
    
# plot image function to check predictions
def show_picture(im_rgb):
    plt.imshow(im_rgb)
    plt.show()
    
# alter image to make the model easier to read
datagen = ImageDataGenerator(
    shear_range = 0.25,
    zoom_range = 0.1,
)

# get all the images in an array
image_path = train_dataset['file_path'].values.astype(str)
i = 0
images = []
while i < len(image_path):
    images.append("../input/com3025-2022-image-classification-challenge/"+image_path[i])
    i+= 1
    
# get labels
labels = train_dataset['label'].values

file_ids = []
i = 0
while i < len(image_path):
    file_ids.append([images[i], labels[i].astype(str)])
    i+=1

y = []
image_data = []

i = 0
while i < len(image_path):
    image_data.append(read_and_resize_image(file_ids[i][0]))
    y.append(file_ids[i][1])
    i += 1    

In [9]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

y = to_categorical(y)
X = image_data

X = np.asarray(X)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state = 42, test_size = 0.1)

In [10]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import optimizers
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Sequential
epochs = 15
batch_size = 40
def create_model():

    model = Sequential()
    
    model.add(ResNet50(include_top = False, pooling = 'avg', weights = 'imagenet', classes=102))
    model.add(Dropout(0.5))
    model.add(Dense(102, activation="softmax"))
    
    

    return model


model = create_model()#adam  rmsprop  categorical     
model.compile(optimizer= optimizers.Adam(learning_rate = 1e-5, clipnorm = 1.), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [11]:
history = model.fit(X_train, y_train, validation_data=(X_valid,y_valid), epochs= 20, batch_size = 40)

In [12]:
import os
X_test = []
prefix = '../input/com3025-2022-image-classification-challenge/test/'

test_files = [(prefix+x) for (x) in os.listdir("../input/com3025-2022-image-classification-challenge/test")]


for test_image_path in tqdm(test_files):
  X_test.append(read_and_resize_image(test_image_path))


X_test = np.asarray(X_test)

In [13]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis = 1)

import re

test_files = [('test/'+x) for (x) in os.listdir("../input/com3025-2022-image-classification-challenge/test")]

print(test_files[0])

sorted_preds = []
for i in range (0, len(X_test)):
    pred =  ((test_files[i]), y_pred[i])
    sorted_preds.append(pred)
 
sorted_preds = sorted(sorted_preds, key=lambda prediction: prediction[0])
with open("submission_sample.csv".format(prefix), "w+", newline = "") as f:
    writer = csv.writer(f)
    writer.writerow(['file_path','label'])
    for i in range (0, len(X_test)):
        writer.writerow([(sorted_preds[i][0]), (sorted_preds[i][1])])

print("Submission_sample.csv updated successfully")